In [2]:
import os 
import pandas as pd
from lifelines.statistics import logrank_test
from lifelines import CoxPHFitter

os.chdir('/cloud-home/U1039935/Autosurv/autostat')

In [3]:
data_t = pd.read_csv('train_test_split/minmax_normalized/data_train_minmax_overall.csv')
data_test = pd.read_csv('train_test_split/minmax_normalized/data_test_minmax_overall.csv')
data = pd.concat([data_t, data_test], ignore_index=True)
data.shape

(1058, 3222)

In [4]:
X = data.iloc[:,7:]
# # Rename columns in X
# X.columns = range(1, len(X.columns)+1)
y = data.iloc[:,1:3]

In [5]:
# Assuming X_train and y_train are your training data
logrank_p_values = {}
for gene in X.columns:
    p_value = logrank_test(X[gene], y['OS.time'], y['OS']).p_value
    logrank_p_values[gene] = p_value



In [7]:
# Select the top N genes based on p-values
selected_genes = [gene for gene, p_value in sorted(logrank_p_values.items(), key=lambda x: x[1])[:16]]
selected_genes

['ENSG00000169764',
 'ENSG00000115380',
 'ENSG00000168216',
 'ENSG00000131080',
 'ENSG00000154734',
 'ENSG00000184005',
 'ENSG00000154727',
 'ENSG00000185010',
 'ENSG00000069431',
 'ENSG00000171056',
 'ENSG00000137693',
 'ENSG00000140285',
 'ENSG00000138686',
 'ENSG00000138640',
 'ENSG00000005020',
 'ENSG00000157557']

In [10]:
newdata = pd.concat([y, data[selected_genes]], axis=1)
newdata.head()

,OS,OS.time,ENSG00000169764,ENSG00000115380,ENSG00000168216,ENSG00000131080,ENSG00000154734,ENSG00000184005,ENSG00000154727,ENSG00000185010,ENSG00000069431,ENSG00000171056,ENSG00000137693,ENSG00000140285,ENSG00000138686,ENSG00000138640,ENSG00000005020,ENSG00000157557
0,0,4005,0.288660,0.251243,0.179178,0.163436,0.288381,0.080484,0.277206,0.151743,0.209378,0.201190,0.283928,0.151464,0.340786,0.471369,0.549853,0.442541
1,0,1474,0.231877,0.409544,0.455783,0.390398,0.254931,0.084961,0.284712,0.132663,0.197239,0.215591,0.319967,0.214973,0.321198,0.091039,0.445270,0.438185
2,0,1448,0.233115,0.493263,0.401942,0.233497,0.356321,0.139806,0.377283,0.169314,0.182895,0.256897,0.601204,0.316897,0.395795,0.131893,0.427862,0.524814
3,0,348,0.190794,0.442827,0.501240,0.414375,0.229168,0.109046,0.457959,0.176684,0.159296,0.187419,0.634947,0.244479,0.395400,0.184385,0.474244,0.425818
4,0,303,0.377800,0.379547,0.279856,0.270377,0.134867,0.011153,0.337339,0.009222,0.100231,0.072721,0.398579,0.011051,0.627937,0.401367,0.107638,0.554267


In [11]:
newdata.shape

(1058, 18)

In [54]:
cph = CoxPHFitter()
cph.fit(newdata, duration_col='OS.time', event_col='OS')

<lifelines.CoxPHFitter: fitted with 1058 total observations, 883 right-censored observations>

In [60]:
c_index = cph.score(newdata, scoring_method="concordance_index")
print(f"Concordance Index: {c_index}")

Concordance Index: 0.7385339885339886


# How to determine N, similiar with the input dim on LFsurv?